# Implementation of F1 driver skill model (Phillips 2014)

In [1]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import sqlalchemy as sql

In [2]:
# find .env automagically by walking up directories until it's found, then
# load up the .env entries as environment variables
load_dotenv(find_dotenv())

True

## 1. Extracting Data from MySQL Database

Database credentials in ENV variables through `dotenv`

In [3]:
ergast_url = sql.URL.create(
    'mysql+pymysql',
    username = os.getenv("ERGASTF1_DB_USERNAME"),
    password = os.getenv("ERGASTF1_DB_PASSWORD"),
    host = os.getenv("ERGASTF1_DB_HOSTNAME"),
    database = os.getenv("ERGASTF1_DB_NAME"),
)

ergast_engine = sql.create_engine(ergast_url, echo = False)

### Results data

In [7]:
query = r"""WITH dnf AS (
    SELECT
        statusId,
        CASE 
            WHEN statusId IN (1, 11, 12, 13, 14, 15, 16, 17, 18, 19, 45, 50, 53, 55, 58, 88, 
                111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 127, 128,
                133, 134) THEN "Finished"
            WHEN statusId IN (2, 3, 4, 20, 62, 68, 77, 81, 85, 90, 96, 97, 100, 130, 139) THEN "Driver DNF"
            ELSE "Non-Driver DNF"
        END AS dnfstatus
    FROM status
    ORDER BY statusId
), 
fullExp AS (
    SELECT
        YEAR(races.date) AS season,
        driverStandings.driverId
    FROM 
        driverStandings LEFT JOIN races ON driverStandings.raceId = races.raceId 
            LEFT JOIN drivers on driverStandings.driverId = drivers.driverId
    GROUP BY YEAR(races.date), driverStandings.driverId
    ORDER BY YEAR(races.date)
),
recentExp AS (
    SELECT
        season,
        driverId,
        COUNT(season) OVER (
            PARTITION BY driverId
            ORDER BY season
            RANGE 3 PRECEDING
        ) AS expRecentYears
    FROM fullExp
    ORDER BY season
),
exp_tbl AS (
    SELECT
        driverStandings.raceId,
        races.date,
        YEAR(races.date) AS season,
        driverStandings.driverId,
        drivers.code,
        COUNT(driverStandings.raceId) OVER (
                PARTITION BY driverStandings.driverId
                ORDER BY races.date ASC
                RANGE INTERVAL 3 YEAR PRECEDING
            ) AS totalRecentRaces,
        recentExp.expRecentYears
    FROM driverStandings LEFT JOIN races ON driverStandings.raceId = races.raceId 
        LEFT JOIN drivers ON driverStandings.driverId = drivers.driverId
        LEFT JOIN recentExp ON (YEAR(races.date) = recentExp.season AND driverStandings.driverId = recentExp.driverId)
    ORDER BY races.date
)
SELECT
    results.resultId,
    results.raceId, 
    races.year AS season, 
    races.date, 
    results.driverId, 
    constructorId,
    grid, 
    positionOrder, 
    results.statusId, 
    dnf.dnfstatus AS dnfStatus,
    drivers.code AS driverCode, 
    drivers.dob,
    (YEAR(races.date) - YEAR(drivers.dob) - (CASE WHEN MONTH(races.date) < MONTH(drivers.dob) THEN 1 ELSE 0 END)) AS driverAge,
    exp_tbl.expRecentYears
FROM 
    results LEFT JOIN dnf ON results.statusId = dnf.statusId
    LEFT JOIN races ON results.raceId = races.raceId
    LEFT JOIN drivers on results.driverId = drivers.driverId
    LEFT JOIN exp_tbl on (results.raceId = exp_tbl.raceId AND results.driverId = exp_tbl.driverId)
ORDER BY date DESC, positionOrder ASC
;"""

In [8]:
resultsdf_raw = pd.read_sql(query, ergast_engine, index_col = 'resultId')
resultsdf_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25900 entries, 25886 to 20045
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   raceId          25900 non-null  int64  
 1   season          25900 non-null  int64  
 2   date            25900 non-null  object 
 3   driverId        25900 non-null  int64  
 4   constructorId   25900 non-null  int64  
 5   grid            25900 non-null  int64  
 6   positionOrder   25900 non-null  int64  
 7   statusId        25900 non-null  int64  
 8   dnfStatus       25900 non-null  object 
 9   driverCode      9319 non-null   object 
 10  dob             25900 non-null  object 
 11  driverAge       25900 non-null  int64  
 12  expRecentYears  25431 non-null  float64
dtypes: float64(1), int64(8), object(4)
memory usage: 2.8+ MB


In [ ]:
def preproc_change_type(df):    
    df['dnfstatus'] = df['dnfstatus'].astype('category')
    df['season'] = df['season'].astype('category')
    df['driverCode'] = df['driverCode'].astype('category')
    df['date'] = pd.to_datetime(df['date'])
    df['dob'] = pd.to_datetime(df['dob'])

    for col_int16 in ['raceId', 'driverId', 'constructorId']:
        df[col_int16] = df[col_int16].astype('uint16')
    
    for col_int8 in ['grid', 'positionOrder', 'driverAge', 'recenrt']:
        df[col_int8] = df[col_int8].astype('uint8')

    return df

def preproc_drop_cols(df):
    df.drop(columns = ['statusId'])
    return df

resultsdf = (resultsdf_raw.copy()
    .pipe(preproc_change_type)
    .pipe(preproc_drop_cols)             
)

In [ ]:
resultsdf.info()

In [ ]:
resultsdf.head()

### Driver data

## 2. Exploration